# Tutorial: syft-perm in 5 mins

## Common Ground
Everyone needs to modify permissions in SyftBox files - whether you're sharing data with specific users, making files public, or restricting access to sensitive information.

## Problem
Modifying `syft.pub.yaml` files manually is:
- **Tricky** - Complex YAML structure with patterns, inheritance, and hierarchy rules
- **Error-prone** - Silent failures when patterns don't match or inheritance doesn't work as expected
- **Time-consuming** - Need to understand glob patterns, permission levels, and file limits

## Solution
**syft-perm** provides utility functions and `SyftFile`/`SyftFolder` classes that make permissions easy:
- Simple Python API for common permission tasks
- Automatic YAML generation and validation
- Clear permission checking and debugging
- Built-in support for file limits and advanced features

## Benefits
SyftBox code becomes easier to write correctly the first time, with fewer bugs and clearer intent.

## Installation & Setup

In [ ]:
# Install syft-perm
# !pip install syft-perm

import syft_perm as sp
from pathlib import Path
import tempfile
import os

# Create a demo workspace
demo_dir = Path(tempfile.mkdtemp(prefix="syft_perm_demo_"))
print(f"Demo workspace: {demo_dir}")

## 1. Basic File Permissions (2 minutes)

The most common use case: grant access to specific files.

In [ ]:
# Create a file and open it with syft-perm
my_file = demo_dir / "secret_data.txt"
my_file.write_text("This is sensitive information")

# Open the file with syft-perm
syft_file = sp.open(my_file)
print(f"File: {syft_file}")

In [ ]:
# Grant different permission levels
syft_file.grant_read_access("alice@example.com", force=True)     # Alice can read
syft_file.grant_write_access("bob@example.com", force=True)      # Bob can read + write
syft_file.grant_admin_access("charlie@example.com", force=True)  # Charlie can do everything

# Check permissions
print("Alice can read:", syft_file.has_read_access("alice@example.com"))
print("Alice can write:", syft_file.has_write_access("alice@example.com"))
print("Bob can write:", syft_file.has_write_access("bob@example.com"))
print("Charlie can admin:", syft_file.has_admin_access("charlie@example.com"))

In [ ]:
# View the generated permissions
syft_file

## 2. Public Access (1 minute)

Make files accessible to everyone.

In [ ]:
# Create a public file
public_file = demo_dir / "public_announcement.txt"
public_file.write_text("This is a public announcement!")

syft_public = sp.open(public_file)
syft_public.grant_read_access("*", force=True)  # "*" means everyone

# Test public access
print("Anyone can read:", syft_public.has_read_access("random@example.com"))
print("Anyone can read:", syft_public.has_read_access("another@user.com"))

syft_public

## 3. Folder Permissions (1 minute)

Apply permissions to entire folders.

In [ ]:
# Create a project folder
project_folder = demo_dir / "team_project"
project_folder.mkdir()

# Create some files in the folder
(project_folder / "README.md").write_text("# Team Project")
(project_folder / "data.csv").write_text("name,value\ntest,123")

# Apply permissions to the entire folder
syft_folder = sp.open(project_folder)
syft_folder.grant_read_access("team@example.com", force=True)
syft_folder.grant_write_access("lead@example.com", force=True)

syft_folder

In [ ]:
# Files in the folder inherit permissions
readme_file = sp.open(project_folder / "README.md")
print("README permissions inherited:")
print("Team can read:", readme_file.has_read_access("team@example.com"))
print("Lead can write:", readme_file.has_write_access("lead@example.com"))

readme_file

## 4. File Limits (1 minute)

Control file size and type restrictions.

In [ ]:
# Create a file with size limits
upload_file = demo_dir / "user_upload.txt"
upload_file.write_text("Small file content")

syft_upload = sp.open(upload_file)

# Grant permission with file size limit (1KB)
syft_upload.grant_create_access("users@example.com", force=True)
syft_upload.set_file_limits(
    max_size=1024,        # 1KB limit
    allow_dirs=False,     # No directories
    allow_symlinks=False  # No symlinks
)

# Check limits
limits = syft_upload.get_file_limits()
print("File limits:", limits)

syft_upload

## 5. Permission Debugging

Understand why permissions work (or don't work).

In [ ]:
# Debug permissions with detailed explanations
explanation = syft_file.explain_permissions("alice@example.com")
print(explanation)

In [ ]:
# Check specific permission with reasons
has_write, reasons = syft_file._check_permission_with_reasons("alice@example.com", "write")
print(f"Alice can write: {has_write}")
print("Reasons:", reasons)

## That's it! 🎉

In 5 minutes, you've learned:
- ✅ Basic file permissions (read, write, admin)
- ✅ Public access with `"*"`
- ✅ Folder permissions and inheritance
- ✅ File limits for size and type control
- ✅ Permission debugging

## Next Steps

For advanced features like:
- Complex glob patterns (`**/*.py`, `docs/**`)
- Terminal nodes and inheritance blocking
- Permission hierarchy and nearest-node algorithms
- Cross-directory file movements
- Pattern specificity and conflict resolution

Check out the **Comprehensive SyftBox Permissions Tutorial Series**!

In [ ]:
# Cleanup demo workspace
import shutil
shutil.rmtree(demo_dir)
print("Demo cleanup complete!")